# Common functions

So far, only contains what is expected to be a heavily reused load-and-parse method.

In [ ]:
public T[] LoadAndParse<T>(string path, Func<string, T> parse = null) {  
    parse = parse ?? (p => (T) Convert.ChangeType(p, typeof(T)));
    var rawInput = System.IO.File.ReadAllLines(path);
    return rawInput
    .Where(p=>!string.IsNullOrWhiteSpace(p))
    .Select(parse)
    .ToArray();
}

# --- Day 1: Not Quite Lisp ---


In [ ]:
{
    var input = System.IO.File.ReadAllText(@"day1_input.txt");
    int? firstNegative = null;
    int currentFloor = 0;
    for(int i = 0; i < input.Length; i++)   {
        if(input[i] == '(')
            currentFloor++;
        else
            currentFloor--;
        if(currentFloor < 0)
            firstNegative ??= (i+1);
    }
    Console.WriteLine($"Last floor: {currentFloor}");
    Console.WriteLine($"Santa first enters the basement on level {firstNegative}");
}

Last floor: 232
Santa first enters the basement on level 1783


# --- Day 2: I Was Told There Would Be No Math ---

In [ ]:
record Prism (int L, int W, int H) {
    //Both methods assume the coordinates are ordered l - w - h smallest to largest dimension.
    public int Area => 3*L*W + 2*W*H + 2*H*L;
    public int RibbonLength => 2*L + 2*W + L*W*H;
};

var input = LoadAndParse<Prism>(@"day2_input.txt", p => {
    var input = p.Split("x")
    .Select(p=>Int32.Parse(p))
    .OrderBy(p=>p)
    .ToArray();
    return new Prism(input[0], input[1], input[2]);
});
    
Console.WriteLine(input.Sum(p=>p.Area));
Console.WriteLine(input.Sum(p=>p.RibbonLength));

1606483
3842356


# --- Day 3: Perfectly Spherical Houses in a Vacuum ---

In [ ]:
var input = System.IO.File.ReadAllText(@"day3_input.txt");

int Visits(IEnumerable<IEnumerable<char>> paths) {
    System.Collections.Generic.HashSet<(int x, int y)> Visited = new();
    Visited.Add((0,0));
    
    foreach (var path in paths){
        (int x, int y) currentLocation = (0,0);
        foreach(char direction in path){
            switch (direction) {
                case '>': currentLocation = (currentLocation.x + 1, currentLocation.y); break;
                case '<': currentLocation = (currentLocation.x - 1, currentLocation.y); break;
                case '^': currentLocation = (currentLocation.x, currentLocation.y + 1); break;
                case 'v': currentLocation = (currentLocation.x, currentLocation.y - 1); break;
            }
            Visited.Add(currentLocation);
        }
    }
    return Visited.Count();   
}

Console.WriteLine(Visits(new[]{input}));

var inputTrails = input.Select((p, i) => (instruction: p, follower: i%2)).ToLookup(p=>p.follower, p=>p.instruction);
Console.WriteLine(Visits(inputTrails));

2592
2360


# --- Day 4: The Ideal Stocking Stuffer ---

In [ ]:
var input = System.IO.File.ReadAllText(@"day4_input.txt");
public int FindPrefixCollision(string target, int prefixZeroCount){
    using (var md5 = System.Security.Cryptography.MD5.Create()){
        int probe = 0;
        byte[] hash;
        
        while(true){
            //Note: Significant allocation overhead could probably be eliminated by in-place manipulating the byte-array.
            var bytes = System.Text.Encoding.ASCII.GetBytes($"{input}{probe}");
            hash = md5.ComputeHash(bytes);
            
            byte test = hash.Take(prefixZeroCount / 2).Aggregate((p, q) => (byte) (p | q));
            if(prefixZeroCount % 2 == 1)
                test |= (byte) (hash[prefixZeroCount/2] >> 4);

            if(test == 0){
                Console.WriteLine($"{prefixZeroCount} zero prefix collision found at probe {probe}. {input}{probe} resolves to MD5 {BitConverter.ToString(hash)}");
                return probe;
            }
            
            probe++;
        }
    }
    
}
FindPrefixCollision(input, 5);
FindPrefixCollision(input, 6);

5 zero prefix collision found at probe 282749. yzbqklnj282749 resolves to MD5 00-00-02-C6-55-DF-77-38-24-6E-88-F6-C1-C4-3E-B7
6 zero prefix collision found at probe 9962624. yzbqklnj9962624 resolves to MD5 00-00-00-4B-34-7B-F4-B3-98-B3-F6-2A-CE-7C-D3-01


# --- Day 5: Doesn't He Have Intern-Elves For This? ---

In [ ]:
var input = LoadAndParse<string>(@"day5_input.txt");

bool IsNice1(string candidate) {
    bool HasDublet() => Enumerable.Range(0, candidate.Length-1).Any(i => candidate[i] == candidate[i+1]);
    bool HasEnoughVowels() => candidate.Count(p => new []{'a', 'e', 'i', 'o', 'u'}.Contains(p)) >= 3;
    bool ContainsStopwords() => (new[]{"ab", "cd", "pq", "xy"}).Any(p=>candidate.Contains(p));
    
    return HasDublet() && HasEnoughVowels() && !ContainsStopwords();
}

Console.WriteLine(input.Count(IsNice1));

bool IsNice2(string candidate) {
    bool HasRepeatingLetterPair() => Enumerable.Range(0, candidate.Length-3).Any(i => candidate.Substring(i+2).Contains(candidate.Substring(i, 2))) ;
    bool ContainsDubletWithChaperone() => Enumerable.Range(0, candidate.Length-2).Any(i => candidate[i] == candidate[i+2]);
        
    return HasRepeatingLetterPair() && ContainsDubletWithChaperone();
}

Console.WriteLine(input.Count(IsNice2));


255
55


# --- Day 6: Probably a Fire Hazard ---

In [ ]:
var inputPattern = new System.Text.RegularExpressions.Regex(@"(?'command'turn on|turn off|toggle) (?'x1'\d+),(?'y1'\d+) through (?'x2'\d+),(?'y2'\d+)");
record Instruction (string Command, int X1, int Y1, int X2, int Y2) {
    public void Apply(bool[,] grid) {
        for (int x = X1; x <= X2; x++)
        for(int y = Y1; y <= Y2; y++) 
        switch(Command){
            case "turn on": grid[x,y] = true; break;
            case "turn off": grid[x,y] = false; break;
            case "toggle": grid[x,y] = !grid[x,y]; break;            
        }
    }
    
    public void Apply(int[,] grid) {
        for (int x = X1; x <= X2; x++)
        for(int y = Y1; y <= Y2; y++) 
        switch(Command){
            case "turn on": grid[x,y]++ ; break;
            case "turn off": grid[x,y] = Math.Max(0, grid[x,y]-1); break;
            case "toggle": grid[x,y] += 2; break;            
        }
    }
    
};
var input = LoadAndParse<Instruction>(@"day6_input.txt", p=>{
    var match = inputPattern.Match(p);
    return new Instruction(match.Groups["command"].Value,
        Int32.Parse(match.Groups["x1"].Value), Int32.Parse(match.Groups["y1"].Value),
        Int32.Parse(match.Groups["x2"].Value), Int32.Parse(match.Groups["y2"].Value));
});

bool[,] grid1 = new bool[1000, 1000];
int[,] grid2 = new int[1000, 1000];
foreach (var instruction in input){
    instruction.Apply(grid1);
    instruction.Apply(grid2);
}

Console.WriteLine(grid1.Cast<bool>().Count(p=>p));
Console.WriteLine(grid2.Cast<int>().Sum());

//Just for fun, lets try rendering the display - turns out it's basically a foggy cloud with no real substance :/
var m = grid2.Cast<int>().Max();
var b = new System.Drawing.Bitmap(1000, 1000);
for(int x = 0; x < 1000; x++) 
for(int y = 0; y < 1000; y++) {
    var lumen = (byte)(250 * grid2[x,y] / m); 
    b.SetPixel(x, y,  System.Drawing.Color.FromArgb(lumen, lumen, lumen));
}
b.Save("BW.bmp");

543903
14687245
